# Prompt tuning

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict
import numpy as np

In [3]:
train_df = pd.read_pickle("train_dataset")
test_df = pd.read_pickle("test_dataset")
val_df = pd.read_pickle("val_dataset")

In [4]:
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'test': Dataset.from_pandas(test_df),
    'unsupervised': Dataset.from_pandas(val_df)
})

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 427
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 92
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 92
    })
})

In [6]:
dataset['train'][7]

{'text': 'So regarding part 1 of the question, I think the lady is behind the door!',
 'label': 4}

## Setup

In [7]:
# Install latest bitsandbytes & transformers, accelerate from source
%pip install -q -U bitsandbytes
# %pip install -q -U git+https://github.com/huggingface/transformers.git
# %pip install -q -U git+https://github.com/huggingface/peft.git
# %pip install -q -U git+https://github.com/huggingface/accelerate.git
%pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
# Other requirements for the demo
%pip install gradio
%pip install sentencepiece
%pip install protobuf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Loading the model

In [8]:
# Required for a meta-llama/Llama-2-13b-chat-hf
from dotenv import load_dotenv
import os

load_dotenv()  # This loads the .env file into the environment

hf_token = os.getenv('HF_TOKEN')

In [9]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer, DataCollatorWithPadding
from accelerate import Accelerator
import evaluate

os.environ["HUGGINGFACE_TOKEN"] = hf_token

model_name = "meta-llama/Llama-2-13b-chat-hf"

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_auth_token=True
)

print(f"Successfully loaded the model {model_name} into memory")

Starting to load the model meta-llama/Llama-2-13b-chat-hf into memory


/d/hpc/home/bi4528/.venv/lib64/python3.11/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Successfully loaded the model meta-llama/Llama-2-13b-chat-hf into memory


In [10]:
adapters_name = 'timdettmers/guanaco-13b'

m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tok = LlamaTokenizer.from_pretrained(model_name, use_auth_token=True)
tok.bos_token_id = 1

stop_token_ids = [0]

tok.pad_token = tok.eos_token
tok.pad_token_id = tok.eos_token_id

m.config.pad_token_id = tok.pad_token_id

accelerator = Accelerator()
m = accelerator.prepare(m)
# print(f"Model and tokenizer are ready and using {accelerator.device}")

/d/hpc/home/bi4528/.venv/lib64/python3.11/site-packages/transformers/tokenization_utils_base.py:1974: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [11]:
torch.cuda.empty_cache()

In [12]:
# import gc 
# gc.collect()

In [13]:
from transformers import StoppingCriteria, StoppingCriteriaList

class CompleteWordCriteria(StoppingCriteria):
    def __init__(self, tokenizer, valid_words):
        self.tokenizer = tokenizer
        self.valid_words = valid_words

    def __call__(self, input_ids, scores):
        # Decode the last generated token to a word
        tokens = self.tokenizer.convert_ids_to_tokens(input_ids[0])
        last_word = tokens[-1]
        # Decode the entire sequence to check the last whole word
        decoded_sequence = self.tokenizer.decode(input_ids[0])
        last_generated_word = decoded_sequence.split()[-1]
        # Check if the last whole word generated is a valid category word
        return last_generated_word in self.valid_words


def generate_text(prompt, max_length=4096, max_new_tokens=5):
    # Configure the device for model execution
    # device = "cuda" if torch.cuda.is_available() else "cpu"
    # m.to(device)

    # Encode the input prompt into tensor format
    inputs = tok(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_length)
    inputs = {k: v.to(m.device, dtype=torch.long)for k, v in inputs.items()}

    # Ensure inputs are on the correct device and dtype is Long
    input_ids = inputs['input_ids'].to(m.device, dtype=torch.long)
    attention_mask = inputs['attention_mask'].to(m.device, dtype=torch.long)

    # Print statements for debugging
    #print(f"input_ids dtype: {input_ids.dtype}")
    #print(f"attention_mask dtype: {attention_mask.dtype}")

    # Define the stopping criteria
    valid_categories = ["Seminar", "Deliberation", "Social", "UX", "Procedure", "Imaginative", "Other"]
    criteria_list = StoppingCriteriaList([CompleteWordCriteria(tok, valid_categories)])

    # Generate text using the model
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            output_sequences = m.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                stopping_criteria=criteria_list
            )

    # Decode the generated ids to text
    text = tok.decode(output_sequences[0], skip_special_tokens=True)
    return text

In [14]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')


/d/hpc/home/bi4528/.venv/lib64/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
from sklearn.cluster import KMeans
import numpy as np

def BERTEmbbedingsKMeans():

    categories = train_df['label'].unique()

    representative_texts = []

    for category in categories:
        # Filter data for the current category
        category_data = train_df[train_df['label'] == category]
        texts = category_data['text'].tolist()
        
        # Generate embeddings
        embeddings = model.encode(texts, show_progress_bar=True)
        
        # Perform K-means clustering
        n_clusters = 5
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        cluster_labels = kmeans.fit_predict(embeddings)
        
        # Add cluster labels back to the DataFrame for the current category
        category_data['cluster_label'] = cluster_labels
        
        # Select representatives from each cluster
        for cluster in range(n_clusters):
            cluster_data = category_data[category_data['cluster_label'] == cluster]
            # Compute distances from the cluster centroids
            centroids = kmeans.cluster_centers_[cluster]
            distances = cluster_data.apply(lambda x: np.linalg.norm(model.encode([x['text']]) - centroids), axis=1)
            chosen_indices = distances.nsmallest(1).index  # Select one representative per cluster
            representative_texts.extend(category_data.loc[chosen_indices].to_dict('records'))
    return representative_texts


In [16]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import ward, fcluster, dendrogram
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import pdist

def BERTEmbeddingsHC():

    representative_texts = []

    for category in train_df['label'].unique():
        # Filter data for the current category and reset index for alignment
        category_data = train_df[train_df['label'] == category].reset_index(drop=True)
        texts = category_data['text'].tolist()

        # Generate embeddings
        embeddings = model.encode(texts, show_progress_bar=True)

        # Compute the condensed distance matrix for hierarchical clustering
        distance_matrix = pdist(embeddings, 'cosine')

        # Perform hierarchical clustering
        linkage_matrix = ward(distance_matrix)

        # Apply clustering to the data
        cluster_labels = fcluster(linkage_matrix, t=5, criterion='maxclust')
        category_data['cluster_label'] = cluster_labels

        # Select representatives from each cluster
        for cluster in range(1, 6):  # Clusters are 1-indexed from fcluster
            cluster_data = category_data[category_data['cluster_label'] == cluster]
            
            if not cluster_data.empty:
                cluster_indices = cluster_data.index.tolist()
                cluster_embeddings = embeddings[cluster_indices]
                centroid = np.mean(cluster_embeddings, axis=0)
                distances = np.linalg.norm(cluster_embeddings - centroid, axis=1)
                min_index = distances.argmin()
                representative_texts.append(cluster_data.iloc[min_index].to_dict())
    return representative_texts


In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

def tfidf_KMeans():
    
    representative_texts = []

    for category in train_df['label'].unique():
        # Filter data for the current category
        category_data = train_df[train_df['label'] == category].copy()

        # Generate TF-IDF vectors for texts within this category
        tfidf_vectorizer = TfidfVectorizer(max_features=1000)
        tfidf_matrix = tfidf_vectorizer.fit_transform(category_data['text'])

        # Perform K-means clustering
        n_clusters = min(7, len(category_data))  # Ensure we don't exceed the number of samples
        if n_clusters > 1:  # Proceed only if clustering is possible
            kmeans = KMeans(n_clusters=n_clusters, random_state=0)
            cluster_labels = kmeans.fit_predict(tfidf_matrix)

            # Add cluster labels back to the category-specific DataFrame
            category_data['cluster_label'] = cluster_labels

            # Select representative texts from each cluster
            for cluster in range(n_clusters):
                cluster_data = category_data[category_data['cluster_label'] == cluster]
                # Pick the text closest to the cluster center as the representative
                if not cluster_data.empty:
                    center = kmeans.cluster_centers_[cluster]
                    distances = cluster_data.apply(lambda x: np.linalg.norm(
                        tfidf_vectorizer.transform([x['text']]).toarray() - center), axis=1)
                    min_index = distances.idxmin()
                    representative_texts.append(cluster_data.loc[min_index].to_dict())
        else:
            # If clustering is not possible, take a random sample
            representative_texts.append(category_data.sample(n=1).to_dict('records')[0])
    return representative_texts

In [18]:
labelToCategory = {
    0: 'Deliberation',
    1: 'Imaginative',
    2: 'Other',
    3: 'Procedure',
    4: 'Seminar',
    5: 'Social',
    6: 'UX'
}

In [19]:
categoryToLabel = {
    'Deliberation': 0,
    'Imaginative': 1,
    'Other': 2,
    'Procedure': 3,
    'Seminar': 4,
    'Social': 5,
    'UX': 6
}


In [20]:
def create_prompt(test_text, representatives):
    prompt = "Classify the text into categories: Seminar, Deliberation, Social, UX, Procedure, Imaginative, Other.\nHere are some annotated examples:\n"
    for example in representatives:
        prompt += f"Text: \'{example['text']}\' - Category: {labelToCategory[example['label']]}\n"
    prompt += f"Text: \'{test_text}\' - Category:"
    return prompt

# Example use of the prompt with a test text
test_text = "Oooo I have another idea! What if the princess sends the man to the tiger and right out tells her father she did it and wants a trial. Then because he loves her he begs her to choose the door with the young man he picks for her, but she chooses the tiger."
representative_texts = BERTEmbbedingsKMeans() #BERTEmbeddingsHC(), tfidf_KMeans()
prompt = create_prompt(test_text, representative_texts)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipykernel_1491297/3720603788.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['cluster_label'] = cluster_labels


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_1491297/3720603788.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['cluster_label'] = cluster_labels


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

/tmp/ipykernel_1491297/3720603788.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['cluster_label'] = cluster_labels


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_1491297/3720603788.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['cluster_label'] = cluster_labels


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_1491297/3720603788.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['cluster_label'] = cluster_labels


In [21]:
print(prompt)

Classify the text into categories: Seminar, Deliberation, Social, UX, Procedure, Imaginative, Other.
Here are some annotated examples:
Text: 'I agree' - Category: Seminar
Text: 'I want to believe the tiger was behind the door because the princess was semi-barbaric but she was also filled with despair and jealousy.  The green eyed monster.. gets you every time...so she knows she has lost him, but she doesn't have to let the Lady have him.' - Category: Seminar
Text: 'As for the second part, I'm not sure if the King would carry out the same punishment for his daughter as he would the commoners. However, he would definitely think of another way to punish her that would consist of a clever, yet barbaric twist for her to prove her loyalty and worth.' - Category: Seminar
Text: 'Hi everyone! I think what happens next is that the princess let her lover marry the woman. The reason why I think she would do that is because she loves the man (as they had made each other the center of each others un

In [22]:
generated_text = generate_text(prompt)
print("Generated Text:", generated_text)

Generated Text: Classify the text into categories: Seminar, Deliberation, Social, UX, Procedure, Imaginative, Other.
Here are some annotated examples:
Text: 'I agree' - Category: Seminar
Text: 'I want to believe the tiger was behind the door because the princess was semi-barbaric but she was also filled with despair and jealousy.  The green eyed monster.. gets you every time...so she knows she has lost him, but she doesn't have to let the Lady have him.' - Category: Seminar
Text: 'As for the second part, I'm not sure if the King would carry out the same punishment for his daughter as he would the commoners. However, he would definitely think of another way to punish her that would consist of a clever, yet barbaric twist for her to prove her loyalty and worth.' - Category: Seminar
Text: 'Hi everyone! I think what happens next is that the princess let her lover marry the woman. The reason why I think she would do that is because she loves the man (as they had made each other the center o

In [23]:
def predict_category(prompt):
    generated_text = generate_text(prompt)
    # Extract the predicted category from the generated text
    return generated_text.split()[-1]

In [24]:
def evaluate_model(test_df, all_examples):
    predictions = []
    for index, row in test_df.iterrows(): 
        prompt = create_prompt(row['text'], all_examples)
        predicted_label = predict_category(prompt)
        predictions.append(predicted_label)
    return predictions


In [25]:
actual_labels = test_df['label'].astype(int).tolist()

In [26]:
def calculate_accuracy(predicted_labels, actual_labels):
    correct_count = sum(p == labelToCategory[a] for p, a in zip(predicted_labels, actual_labels))
    accuracy = correct_count / len(actual_labels)
    return accuracy


In [27]:
predicted_labels = evaluate_model(test_df, BERTEmbbedingsKMeans())
accuracy = calculate_accuracy(predicted_labels, actual_labels)
print(f"BERT embeddings with Kmeans representatives accuracy: {accuracy * 100:.2f}%")

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipykernel_1491297/3720603788.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['cluster_label'] = cluster_labels


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_1491297/3720603788.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['cluster_label'] = cluster_labels


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_1491297/3720603788.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['cluster_label'] = cluster_labels


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_1491297/3720603788.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_data['cluster_label'] = cluster_labels


BERT embeddings with Kmeans representatives accuracy: 1.09%


In [28]:
predicted_labels = evaluate_model(test_df, BERTEmbeddingsHC())
accuracy = calculate_accuracy(predicted_labels, actual_labels)
print(f"BERT embeddings with hierachical clustering representatives accuracy: {accuracy * 100:.2f}%")

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

BERT embeddings with hierachical clustering representatives accuracy: 1.09%


In [29]:
predicted_labels = evaluate_model(test_df, tfidf_KMeans())
accuracy = calculate_accuracy(predicted_labels, actual_labels)
print(f"TF-IDF with Kmeans representatives accuracy: {accuracy * 100:.2f}%")

TF-IDF with Kmeans representatives accuracy: 1.09%


Zero-shot prompting

In [30]:
category_description = """You will be given a short social media post which you need to classify into one of the following categories: Seminar, Deliberation, Social, UX, Procedure, Imaginative, Other.
Here is a description and example of each category, based on which each post is classified:
Seminar: Posts discussing the deeper meanings of content, encouraging analysis and interpretation.
Deliberation: Posts about making decisions, often involving questions and considerations for future actions.
Social: Posts to establish or maintain relationships, often casual and friendly in nature.
UX: Posts discussing the user experience, including issues and feedback about interfaces and usability.
Procedure: Posts about accomplishing a task, often with step-by-step instructions or suggestions for organizing activities.
Imaginative: Posts about imaginative content, often involving hypothetical scenarios or creative storytelling.
Other: Posts that do not fit into any of the above categories.
Post: "Also, I got an email letting us know that Elias Royal is also a part of our group."
Category:"""

In [31]:
category_description = """You will be given a short social media post which you need to classify into one of the following categories: Seminar, Deliberation, Social, UX, Procedure, Imaginative, Other.
Here is a description of each category, based on which each post is classified:

Posts discussing the deeper meanings of content, encouraging analysis and interpretation.
Category: Seminar

Posts about making decisions, often involving questions and considerations for future actions.
Category: Deliberation

Posts to establish or maintain relationships, often casual and friendly in nature.
Category: Social

Posts discussing the user experience, including issues and feedback about interfaces and usability.
Category: UX

Posts about accomplishing a task, often with step-by-step instructions or suggestions for organizing activities.
Category: Procedure

Posts about imaginative content, often involving hypothetical scenarios or creative storytelling.
Category: Imaginative

Posts that do not fit into any of the above categories.
Category: Other

Post: "Also, I got an email letting us know that Elias Royal is also a part of our group."
Category:"""

In [32]:
generated_text = generate_text(category_description)
print("Generated Text:", generated_text)

Generated Text: You will be given a short social media post which you need to classify into one of the following categories: Seminar, Deliberation, Social, UX, Procedure, Imaginative, Other.
Here is a description of each category, based on which each post is classified:

Posts discussing the deeper meanings of content, encouraging analysis and interpretation.
Category: Seminar

Posts about making decisions, often involving questions and considerations for future actions.
Category: Deliberation

Posts to establish or maintain relationships, often casual and friendly in nature.
Category: Social

Posts discussing the user experience, including issues and feedback about interfaces and usability.
Category: UX

Posts about accomplishing a task, often with step-by-step instructions or suggestions for organizing activities.
Category: Procedure

Posts about imaginative content, often involving hypothetical scenarios or creative storytelling.
Category: Imaginative

Posts that do not fit into any

Few-shot prompting

In [33]:
category_description = """You will be given a short social media post which you need to classify into one of the following categories: Seminar, Deliberation, Social, UX, Procedure, Imaginative, Other.
Here is a description and example of each category, based on which each post is classified:

Posts discussing the deeper meanings of content, encouraging analysis and interpretation.
Category: Seminar

Posts about making decisions, often involving questions and considerations for future actions.
Category: Deliberation

Posts to establish or maintain relationships, often casual and friendly in nature.
Category: Social

Posts discussing the user experience, including issues and feedback about interfaces and usability.
Category: UX

Posts about accomplishing a task, often with step-by-step instructions or suggestions for organizing activities.
Category: Procedure

Posts about imaginative content, often involving hypothetical scenarios or creative storytelling.
Category: Imaginative

Posts that do not fit into any of the above categories.
Category: Other

Here are examples of posts and their categories:
Post: I found myself conflicted for the princess. I always want a happy ending of some kind in the stories I read. However, it seems the princess loses either way in this story. If I were in her place, I would have allowed him to marry the woman rather than have him die a painful and bloody death.
Category: Imaginative

Post: Unfortunately I can't click in the box below at all, so it's anyone's guess what style our response is supposed to be in. I would imagine answering the questions simply would be fine.
Category: UX

Post: Dr. Austin said have fun with it, and it is fun to ponder what might happen in the story. I think the main reason for setting a time to discuss it at the same time would be to test the collaborative software, using the box below instead of this one. I believe we could all write in it at the same time and see each other typing and change each other's words until we all agree we are done with our assessment. If we are able to do that, we might find there are unexpected positives or negatives to the collaboration process or system that we can only otherwise surmise. As you say, we've all responded to the prompts, so what we say doesn't matter that much, so all we'd need is a few minutes to see how using the box works. So if you want to try meeting, I could do Wednesday any time of day. Just set a time and I'll log on then. Thanks.
Category: Procedure

Post: I could write the first draft based on my interpretation of our answers, then we could take turns editing - round robin style until we are happy with the result
Category: Deliberation

Post: Curiosity got the best of me, and I googled this book, which I'd never heard of despite it being a well-known allegory. It presents an unsolvable problem, yet we all felt pretty confident taking a stab at answering the prompts. I wonder what we are to make of all of us feeling pretty certain the princess chose the tiger. Is our choice based on the author's characterization of her or our belief that human nature is fundamentally selfish?
Category: Seminar

Post: I apologize again that I couldn't be part of the original discussion. It was clearly productive and I'm disappointed to have missed out on the opportunity.
Category: Social

"""

In [34]:
generated_text = generate_text(category_description)
print("Generated Text:", generated_text)

Generated Text: You will be given a short social media post which you need to classify into one of the following categories: Seminar, Deliberation, Social, UX, Procedure, Imaginative, Other.
Here is a description and example of each category, based on which each post is classified:

Posts discussing the deeper meanings of content, encouraging analysis and interpretation.
Category: Seminar

Posts about making decisions, often involving questions and considerations for future actions.
Category: Deliberation

Posts to establish or maintain relationships, often casual and friendly in nature.
Category: Social

Posts discussing the user experience, including issues and feedback about interfaces and usability.
Category: UX

Posts about accomplishing a task, often with step-by-step instructions or suggestions for organizing activities.
Category: Procedure

Posts about imaginative content, often involving hypothetical scenarios or creative storytelling.
Category: Imaginative

Posts that do not 

In [35]:
def create_static_prompt(test_text, prompt):
    prompt += f"Post: \'{test_text}\'\nCategory:"
    return prompt

In [36]:
print(create_static_prompt("then click submit", category_description))

You will be given a short social media post which you need to classify into one of the following categories: Seminar, Deliberation, Social, UX, Procedure, Imaginative, Other.
Here is a description and example of each category, based on which each post is classified:

Posts discussing the deeper meanings of content, encouraging analysis and interpretation.
Category: Seminar

Posts about making decisions, often involving questions and considerations for future actions.
Category: Deliberation

Posts to establish or maintain relationships, often casual and friendly in nature.
Category: Social

Posts discussing the user experience, including issues and feedback about interfaces and usability.
Category: UX

Posts about accomplishing a task, often with step-by-step instructions or suggestions for organizing activities.
Category: Procedure

Posts about imaginative content, often involving hypothetical scenarios or creative storytelling.
Category: Imaginative

Posts that do not fit into any of 

In [37]:
def evaluate_static_model(test_df, prompt):
    predictions = []
    for index, row in test_df.iterrows(): 
        prompt = create_static_prompt(row['text'], prompt)
        predicted_label = predict_category(prompt)
        predictions.append(predicted_label)
    return predictions

In [38]:
predicted_labels = evaluate_static_model(test_df, category_description)
accuracy = calculate_accuracy(predicted_labels, actual_labels)
print(f"Few-shot prompting representatives accuracy: {accuracy * 100:.2f}%")